::::
:::{thebe-button}
:::
::::

# Get thermal data


In [ ]:
from functools import cache

from boilercore.fits import Fit, fit_from_params
from boilercore.models.geometry import GEOMETRY
from devtools import pprint
from matplotlib.pyplot import subplots
from pandas import DataFrame, Series, concat, read_csv
from seaborn import lineplot, move_legend, scatterplot

from boilercv_docs.nbs import init
from boilercv_pipeline.stages.common.e230920 import const
from boilercv_pipeline.stages.common.e230920.types import DfNbOuts
from boilercv_pipeline.stages.get_thermal_data import Deps
from boilercv_pipeline.stages.get_thermal_data import GetThermalData as Params

PARAMS = None
C = const.thermal_cols

FIT = Fit()
M_TO_CM = 1e-2
M2_TO_CM2 = M_TO_CM**2
WINDOW = 60

In [ ]:
if isinstance(PARAMS, str):
    params = Params.model_validate_json(PARAMS)
else:
    params = Params(context=(_ctx := init()), deps=Deps(context=_ctx))
params.format.set_display_options()
pprint(params)

In [ ]:
MODELS = FIT.get_models(params.deps.modelfunctions)[0]


@cache
def fit(ser: tuple[float, ...]) -> tuple[dict[str, float], dict[str, float]]:
    """Fit for each set of temperatures."""
    return fit_from_params(MODELS, FIT, GEOMETRY.rods["R"], ser)


def apply_fit(df: DataFrame) -> "Series[float]":
    """Fit model function across sample temperatures."""
    return df.loc[:, [c.dst for c in C.sample_temps]].apply(
        lambda ser: fit(tuple(ser))[0]["q_s"] * C.flux.scale, axis="columns"
    )

In [ ]:
sources = [c.src for c in C.sources]
src = concat([
    read_csv(
        p,
        usecols=[C.time.src, *sources],
        parse_dates=[C.time.src],
        index_col=C.time.src,
    )
    for p in params.deps.thermal_paths
]).rename(columns={c.src: c.dst for c in C.sources})
src.head()

In [ ]:
sources = [c.src for c in C.sources]
outs = DfNbOuts(
    df=(
        src.resample("s")
        .mean()
        .reset_index()
        .rename(columns={c.src: c.dst for c in C.sources})
        .ffill()
        .assign(**{
            C.time_elapsed.dst: lambda df: (
                (df[C.time.dst] - df[C.time.dst][0]).dt.total_seconds()
                * C.time_elapsed.scale
            ),
            C.water_temp.dst: lambda df: df[[c.dst for c in C.water_temps]].mean(
                axis="columns"
            ),
            C.boiling.dst: lambda df: df[C.water_temp.dst].max(),
            C.superheat.dst: lambda df: df[C.surface_temp.dst] - df[C.boiling.dst],
            C.subcool.dst: lambda df: df[C.boiling.dst] - df[C.water_temp.dst],
            C.flux.dst: apply_fit,
        })
        .loc[:, [c.dst for c in C.dests]]
    )
)
outs.df.set_index(C.time.dst)[[c.dst for c in C.dests if c not in C.sources]].head()

In [ ]:
latex = (
    outs.df.set_index(C.time.dst)
    .resample("10s")
    .mean()
    .reset_index()
    .rename(columns={c.dst: c.latex for c in C.dests})
)
figure, ax = subplots()
lineplot(
    ax=ax,
    data=latex.set_index(C.time_elapsed.latex)[[C.subcool.latex, C.superheat.latex]],
    dashes=False,
    errorbar=None,
)
ax.set_ylabel(C.subcool.ylabel)
params.format.move_legend(ax)

In [ ]:
figure, ax = subplots()
scatterplot(
    ax=ax,
    data=latex,
    x=C.subcool.latex,
    y=C.flux.latex,
    hue=latex[C.time_elapsed.latex],
)
move_legend(ax, loc="lower center", bbox_to_anchor=(0.5, 1.0), ncol=3)

In [ ]:
figure, ax = subplots()
scatterplot(
    ax=ax,
    data=latex.set_index(C.time_elapsed.latex),
    x=C.superheat.latex,
    y=C.flux.latex,
    hue=C.time_elapsed.latex,
)
move_legend(ax, loc="lower center", bbox_to_anchor=(0.5, 1.0), ncol=3)